In [8]:
import json
import numpy as np
import pandas as pd
import os

In [27]:

#jason datei kleiner machen 

# Pfad zur Eingangs-JSON-Datei
input_file_path = 'data/300k_books Kopie.json'
# Pfad zur Ausgabedatei
output_file_path = 'data/first_30_complete_entries.json'

# Funktion zum Überprüfen, ob ein Eintrag vollständig ist
def is_entry_complete(entry):
    required_fields = ['isbn', 'text_reviews_count', 'country_code', 'language_code', 'average_rating', 'similar_books', 'description', 'format', 'link', 'authors', 'publisher', 'num_pages', 'publication_year', 'book_id', 'ratings_count', 'title']
    return all(field in entry and entry[field] not in (None, "", []) for field in required_fields)

# Liste zum Speichern der vollständigen Einträge
complete_entries = []

# Lesen der JSON-Datei
with open(input_file_path, 'r') as file:
    for line in file:
        entry = json.loads(line.strip())
        if is_entry_complete(entry):
            complete_entries.append(entry)
            if len(complete_entries) == 50000:  # Nur die ersten 30 Einträge speichern
                break

# Schreiben der vollständigen Einträge in die neue JSON-Datei
with open(output_file_path, 'w') as output_file:
    for entry in complete_entries:
        json.dump(entry, output_file)
        output_file.write('\n')

print(f"Die ersten 50000 vollständigen Einträge wurden in '{output_file_path}' gespeichert.")


Die ersten 50000 vollständigen Einträge wurden in 'data/first_30_complete_entries.json' gespeichert.


In [28]:
#jason datei impotieren und schauen ob alles passt 

data = []  # Initialize an empty list to store the JSON objects
with open('data/first_30_complete_entries.json', 'r', encoding='utf-8') as file:
    for line in file:
        # Append each JSON object to the list
        data.append(json.loads(line.strip()))  # Use strip() to remove any leading/trailing whitespace

# Now, data should be a list of dictionaries
print(type(data))  # Should be <class 'list'>
print(data[:2])    # Should display the first two book dictionaries


<class 'list'>
[{'isbn': '0743294297', 'text_reviews_count': '3282', 'series': [], 'country_code': 'US', 'language_code': 'eng', 'popular_shelves': [{'count': '7615', 'name': 'to-read'}, {'count': '728', 'name': 'chick-lit'}, {'count': '673', 'name': 'currently-reading'}, {'count': '404', 'name': 'fiction'}, {'count': '152', 'name': 'books-i-own'}, {'count': '119', 'name': 'jennifer-weiner'}, {'count': '82', 'name': 'chicklit'}, {'count': '76', 'name': 'owned'}, {'count': '72', 'name': 'book-club'}, {'count': '72', 'name': 'friendship'}, {'count': '55', 'name': 'adult-fiction'}, {'count': '54', 'name': 'chic-lit'}, {'count': '51', 'name': 'favorites'}, {'count': '50', 'name': 'adult'}, {'count': '43', 'name': 'contemporary'}, {'count': '39', 'name': 'read-in-2010'}, {'count': '37', 'name': 'library'}, {'count': '35', 'name': 'read-in-2009'}, {'count': '34', 'name': 'owned-books'}, {'count': '33', 'name': 'default'}, {'count': '29', 'name': 'women-s-fiction'}, {'count': '27', 'name': 'a

In [29]:
import json
import numpy as np
import os

# Angenommen, `data` ist deine große JSON-Datei, die bereits in eine Liste von Dictionaries geladen wurde.

# Step 1: Create a set of all unique book IDs in the dataset
all_book_ids = {buch['book_id'] for buch in data}

# Step 2: Create a mapping of book IDs to their indices
book_id_to_index = {book_id: index for index, book_id in enumerate(all_book_ids)}

# Step 3: Prepare a dictionary to hold the filtered books and multi-label targets
filtered_books_dict = {}
multi_label_targets = []

# Prepare a list to track missing similar books
missing_similar_books = []

# Process each book in the dataset
for buch in data:
    # Sicherstellen, dass 'similar_books' vorhanden ist und eine Liste ist
    if 'similar_books' not in buch or not isinstance(buch['similar_books'], list):
        continue
    
    # Filter similar_books to retain only those that exist in all_book_ids
    valid_similar_books = [sim_book for sim_book in buch['similar_books'] if sim_book in all_book_ids]

    # Nur das Buch zu filtered_books_dict hinzufügen, wenn mindestens ein gültiger similar_book vorhanden ist
    if valid_similar_books:
        filtered_books_dict[buch['book_id']] = {
            'title': buch.get('title'),
            'authors': [author['author_id'] for author in buch.get('authors', [])],
            'description': buch.get('description'),
            'similar_books': valid_similar_books  # Behalte nur gültige similar book IDs
        }

        # Erstelle einen multi-hot kodierten Vektor für die gültigen similar_books
        target_vector = np.zeros(len(all_book_ids), dtype=int)
        for similar_book_id in valid_similar_books:
            target_vector[book_id_to_index[similar_book_id]] = 1
        multi_label_targets.append(target_vector)

# Ensure the directory exists
output_dir = 'data'
os.makedirs(output_dir, exist_ok=True)

# Save the filtered books to a JSON file
output_file_path = os.path.join(output_dir, 'filtered_books.json')
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(filtered_books_dict, file, ensure_ascii=False, indent=4)

# Output statistics
print(f"Total books processed: {len(data)}")
print(f"Filtered books: {len(filtered_books_dict)}")


Total books processed: 38633
Filtered books: 22582


In [30]:
import json
import pandas as pd

# Open the JSON file and load the data
with open('data/filtered_books.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

rows = []

# Step 1: Create a set of all unique book IDs in the dataset
all_book_ids = set(data.keys())

for book_id, book_info in data.items():
    # Handle multiple similar books
    for similar_book in book_info.get('similar_books', []):
        # Only create a new row if the similar book ID exists in all_book_ids
        if similar_book in all_book_ids:
            rows.append({
                'book_id': book_id,
                'title': book_info['title'],
                'authors': ', '.join(book_info.get('authors', [])),  # Join authors if there are multiple
                'description': book_info['description'],
                'similar_book': similar_book
            })

# Create a DataFrame from the rows
df = pd.DataFrame(rows)

# Save to CSV
df.to_csv('data/books.csv', index=False)

# Output the number of rows saved
print(f"Total rows saved: {len(df)}")


Total rows saved: 23559


In [31]:
df = pd.read_csv('data/books.csv')
df.info


<bound method DataFrame.info of         book_id                          title                 authors  \
0         89378                     Dog Heaven                    5411   
1       2008910          Buenos días, tristeza                 1357577   
2      18628480                         Stoner  51229, 110982, 5768434   
3      18628480                         Stoner  51229, 110982, 5768434   
4      18628480                         Stoner  51229, 110982, 5768434   
...         ...                            ...                     ...   
23554    836042                           1984           3706, 4464400   
23555    344972  The Life and Times of Chaucer                  481146   
23556   9884033               The Killing Hour            18282, 62817   
23557   9884033               The Killing Hour            18282, 62817   
23558  17779550         Last Train to Istanbul        2882488, 7213767   

                                             description  similar_book  
0     

In [44]:
book_ids = set(df['book_id'])
similar_book_ids = set(df['similar_book'])

# Step 2: Find missing similar book IDs
missing_books = similar_book_ids - book_ids
missing_count = len(missing_books)

# Step 3: Check if every similar book ID is present in the book ID set
all_similar_in_books = similar_book_ids.issubset(book_ids)

print("Alle ähnlichen Buch-IDs sind in den Buch-IDs vorhanden:", all_similar_in_books)
print("Anzahl der fehlenden Bücher:", missing_count)
print("Fehlende Buch-IDs:", missing_books)

Alle ähnlichen Buch-IDs sind in den Buch-IDs vorhanden: True
Anzahl der fehlenden Bücher: 0
Fehlende Buch-IDs: set()


In [45]:
# Step 1: Erstelle eine Menge aller vorhandenen Buch-IDs und ähnlichen Buch-IDs
book_ids = set(df['book_id'])
similar_book_ids = set(df['similar_book'])

# Step 2: Finde die fehlenden ähnlichen Buch-IDs
missing_books = similar_book_ids - book_ids

# Step 3: Droppen der Zeilen, bei denen die similar_book-ID in missing_books enthalten ist
df = df[~df['similar_book'].isin(missing_books)]

# Ausgabe der Anzahl der verbleibenden Zeilen
print(f"Anzahl der verbleibenden Zeilen nach dem Droppen: {len(df)}")

# Optional: Speichern der gefilterten Daten in eine neue CSV-Datei
df.to_csv('data/filtered_books_fina1.csv', index=False)


Anzahl der verbleibenden Zeilen nach dem Droppen: 19025


# Check NAs


In [46]:
missing_description_count = df['description'].isnull().sum()
print(missing_description_count)


0


In [47]:
total_missing = df.isnull().sum().sum()
print(f"Gesamtanzahl der fehlenden Werte: {total_missing}")


Gesamtanzahl der fehlenden Werte: 0


In [48]:
missing_per_column = df.isnull().sum()
print(missing_per_column)


book_id         0
title           0
authors         0
description     0
similar_book    0
dtype: int64


# types checken 

In [49]:
df.dtypes


book_id          int64
title           object
authors         object
description     object
similar_book     int64
dtype: object

# Stop words 

In [51]:
df = pd.read_csv('data/filtered_books_fina1.csv')


In [52]:
import spacy
import pandas as pd

# Lade das englische Modell
nlp = spacy.load('en_core_web_sm')

# Annahme: df ist dein DataFrame
# Entferne Stoppwörter aus der 'description'-Spalte
df['description'] = df['description'].apply(lambda x: ' '.join([token.text for token in nlp(x) if not token.is_stop]))

# Entferne Stoppwörter aus der 'title'-Spalte
df['title'] = df['title'].apply(lambda x: ' '.join([token.text for token in nlp(x) if not token.is_stop]))

# Speichere die bearbeiteten Daten in einer neuen CSV-Datei


In [53]:
print(df['title'].apply(type).value_counts())


non_string_entries = df[~df['title'].apply(lambda x: isinstance(x, str))]
print(non_string_entries)

df['title'] = df['title'].astype(str)

empty_count = (df['title'] == '').sum()
print(f'Number of empty strings in title: {empty_count}')

df['title'] = df['title'].apply(lambda x: ' '.join([token.text for token in nlp(x) if not token.is_stop]) if isinstance(x, str) else '')


title
<class 'str'>    19025
Name: count, dtype: int64
Empty DataFrame
Columns: [book_id, title, authors, description, similar_book]
Index: []
Number of empty strings in title: 57


In [54]:
# Remove rows where 'title' is an empty string
df = df[df['title'] != '']

# Check the number of empty strings in 'title' after removal
num_empty_titles = df['title'].eq('').sum()
print(f'Number of empty strings in title after removal: {num_empty_titles}')


Number of empty strings in title after removal: 0


In [55]:
unique_book_ids = df['book_id'].nunique()  # Get the number of unique book IDs
print(unique_book_ids)  # Print the count of unique book IDs


13154


In [56]:
df.to_csv('data/filtered_books_fina1.csv', index=False)
